<a href="https://colab.research.google.com/github/Ferrariagustinpablo/Data-Analytics-in-R/blob/main/Importing_in_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [1]:
# Load
library(dplyr)
library(ggplot2)
library(tidyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
install.packages("data.table")
library(data.table)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




# Flat files

# CSV: read.csv

In [ ]:
# Import swimming_pools.csv: pools
pools <- read.csv("swimming_pools.csv")

# Print the structure of pools
str(pools)

## stringsAsFactors

With stringsAsFactors, you can tell R whether it should convert strings in the flat file to factors.

In [ ]:
pools <- read.csv("swimming_pools.csv", stringsAsFactors = FALSE)

# TSV: read.delim()

In [ ]:
read.delim("hotdogs.txt", header = FALSE)

# OTHER delimiters: read.table

In [ ]:
# Also for any delimiter
read.table(path, sep = "\t", 
                      col.names = c("type", "calories", "sodium"))

# Other library for importing is: readr

# Efficient library for data.tables: fread()

Works better and is super fast

Also infers column types and separators

In [ ]:
install.packages("data.table")
library(data.table)

fread("potatoes.csv")

In [ ]:
# Import columns 6 and 8 of potatoes.csv: potatoes
potatoes <- fread("potatoes.csv", select = c(6, 8))

# Importing Excel: readxl

In [ ]:
install.packages("readxl")
library(readxl)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# Print the names of all worksheets
excel_sheets("urbanpop.xlsx")


In [ ]:
# Read the sheets, one by one
pop_1 <- read_excel("urbanpop.xlsx", sheet = 1)
pop_2 <- read_excel("urbanpop.xlsx", sheet = 2)

In [ ]:
# Read all Excel sheets with lapply(): pop_list
pop_list <- lapply(excel_sheets("urbanpop.xlsx"), read_excel, path = "urbanpop.xlsx")

## name columns

In [ ]:
# Import the first Excel sheet of urbanpop_nonames.xlsx (specify col_names): pop_b

cols <- c("country", paste0("year_", 1960:1966))

pop_b <- read_excel("urbanpop_nonames.xlsx", col_names = cols)

In [ ]:
# Import the second sheet of urbanpop.xlsx, skipping the first 21 rows: urbanpop_sel
urbanpop_sel <- read_excel("urbanpop.xlsx", sheet = 2, col_names = FALSE, skip = 21)

# Another package powerfull for coding in R and changing in excel: XLConnect

# Importing from databases RMySQL

In [ ]:
install.packages("RMySQL")
library(DBI) # library(RMySQL) not requiered

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



## Establish a connection

The first step to import data from a SQL database is creating a connection to it. 

dbConnect() creates a connection between your R session and a SQL database. The first argument has to be a DBIdriver object, that specifies how connections are made and how data is mapped between R and the database. Specifically for MySQL databases, you can build such a driver with RMySQL::MySQL().

In [ ]:
# Edit dbConnect() call
con <- dbConnect(RMySQL::MySQL(), 
                 dbname = "tweater", 
                 host = "courses.csrrinzqubik.us-east-1.rds.amazonaws.com", 
                 port = 3306,
                 user = "student",
                 password = "datacamp")

you've successfully created the database connection

## List and import tables

In [ ]:
dbListTables(con)

[1] "comments" "tweats"   "users"

In [ ]:
# Import the users table from tweater: users
users <- dbReadTable(con, "users")

# Print users
head(users,2)

,id,name,login
,<dbl>,<chr>,<chr>
1,1,elisabeth,elismith
2,2,mike,mikey


Next to the users, we're also interested in the tweats and comments tables. However, separate dbReadTable() calls for each and every one of the tables in your database would mean a lot of code duplication

## Import all tables 

In [ ]:
# Get table names
table_names <- dbListTables(con)

# Import all tables
tables <- lapply(table_names, dbReadTable, conn = con)

# Print out tables
tables

id,tweat_id,user_id,message
<dbl>,<dbl>,<dbl>,<chr>
1022,87,7,nice!
1000,77,7,great!
1011,49,5,love it
1012,87,1,awesome! thanks!
1010,88,6,yuck!
1026,77,4,not my thing!
1004,49,1,this is fabulous!
1030,75,6,so easy!
1025,88,2,oh yes


## Direct query from R with SQL

In [ ]:
# After connecting database:
elisabeth <- dbGetQuery(con, 
               "SELECT tweat_id FROM comments WHERE user_id = 1")

In [ ]:
# Import post column of tweats where date is higher than '2015-09-21': latest
latest <- dbGetQuery(con, "SELECT post FROM tweats WHERE date > '2015-09-21'")

# Print latest
latest

post
<chr>
open and crush avocado. add shrimps. perfect starter.
"nachos. add tomato sauce, minced meat and cheese. oven for 10 mins."
just eat an apple. simply and healthy.


## Querying in chunks

This is tedious to write, but it gives you the ability to fetch the query's result in chunks rather than all at once. You can do this by specifying the n argument inside dbFetch().

remember this technique when you're struggling with huge databases

In [ ]:
# Send query to the database
res <- dbSendQuery(con, "SELECT * FROM comments WHERE user_id > 4")

# Use dbFetch() twice
dbFetch(res, n = 2)
dbFetch(res)

id,tweat_id,user_id,message
<dbl>,<dbl>,<dbl>,<chr>
1022,87,7,nice!
1000,77,7,great!


id,tweat_id,user_id,message
<dbl>,<dbl>,<dbl>,<chr>
1011,49,5,love it
1010,88,6,yuck!
1030,75,6,so easy!


In [ ]:
# Clear res
dbClearResult(res)

[1] TRUE

# Import flat files from the web


## http link .csv

In [ ]:
# Load the readr package
library(readr)

# Import the csv file: pools
url_csv <- "http://s3.amazonaws.com/assets.datacamp.com/production/course_1478/datasets/swimming_pools.csv"
pools <- read_csv(url_csv)

# Import the txt file: potatoes
url_delim <- "http://s3.amazonaws.com/assets.datacamp.com/production/course_1478/datasets/potatoes.txt"
potatoes <- read_tsv(url_delim)

# Print pools and potatoes
pools
potatoes

## https link .csv

In [6]:
# https URL to the swimming_pools csv file.
url_csv <- "https://s3.amazonaws.com/assets.datacamp.com/production/course_1478/datasets/swimming_pools.csv"

# Import the file using read.csv(): pools1
pools1 <- read.csv(url_csv)
str(pools1)

'data.frame':	20 obs. of  4 variables:
 $ Name     : chr  "Acacia Ridge Leisure Centre" "Bellbowrie Pool" "Carole Park" "Centenary Pool (inner City)" ...
 $ Address  : chr  "1391 Beaudesert Road, Acacia Ridge" "Sugarwood Street, Bellbowrie" "Cnr Boundary Road and Waterford Road Wacol" "400 Gregory Terrace, Spring Hill" ...
 $ Latitude : num  -27.6 -27.6 -27.6 -27.5 -27.4 ...
 $ Longitude: num  153 153 153 153 153 ...


## http link .xls

gdata can handle .xls files that are on the internet. readxl can't, at least not yet. The URL with which you'll be working is already available in the sample code. You will import it once using gdata and once with the readxl package via a workaround.

readxl is not (yet?) able to deal with Excel files that are on the web. However, a simply workaround with download.file() fixes this.

In [8]:
install.packages("gdata")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘gtools’




### gdata or readxls after download.file()

In [10]:
# Load the readxl and gdata package
library(readxl)
library(gdata)

# Specification of url: url_xls
url_xls <- "http://s3.amazonaws.com/assets.datacamp.com/production/course_1478/datasets/latitude.xls"

# Import the .xls file with gdata: excel_gdata
excel_gdata <- read.xls(url_xls)

# Download file behind URL, name it local_latitude.xls
download.file(url_xls, destfile = "local_latitude.xls")

# Import the local .xls file with readxl: excel_readxl
excel_readxl <- read_excel("local_latitude.xls")
str(excel_readxl)

tibble [246 × 2] (S3: tbl_df/tbl/data.frame)
 $ country: chr [1:246] "Afghanistan" "Akrotiri and Dhekelia" "Albania" "Algeria" ...
 $ 1700   : num [1:246] 34.6 34.6 41.3 36.7 -14.3 ...


## https link .RData

There's more: with download.file() you can download any kind of file from the web, using HTTP and HTTPS: images, executable files, but also .RData files. An RData file is very efficient format to store R data.

You can load data from an RData file using the load() function

In [ ]:
# https URL to the wine RData file.
url_rdata <- "https://s3.amazonaws.com/assets.datacamp.com/production/course_1478/datasets/wine.RData"

# Download the wine file to your working directory
download.file(url_rdata, destfile = "wine_local.RData")

# Load the wine data into your workspace using load()
load("wine_local.RData")

# Print out the summary of the wine data
summary(wine)

## http link JSON

In [14]:
# Load the httr package
library(httr)

# Get the url
url <- "http://www.omdbapi.com/?apikey=72bc447a&t=Annie+Hall&y=&plot=short&r=json"
resp <- GET(url)

# Print resp
resp

# Print content of resp as text
content(resp, as = "text")


Response [http://www.omdbapi.com/?apikey=72bc447a&t=Annie+Hall&y=&plot=short&r=json]
  Date: 2021-04-05 15:59
  Status: 200
  Content-Type: application/json; charset=utf-8
  Size: 1.05 kB


[1] "{\"Title\":\"Annie Hall\",\"Year\":\"1977\",\"Rated\":\"PG\",\"Released\":\"20 Apr 1977\",\"Runtime\":\"93 min\",\"Genre\":\"Comedy, Romance\",\"Director\":\"Woody Allen\",\"Writer\":\"Woody Allen, Marshall Brickman\",\"Actors\":\"Woody Allen, Diane Keaton, Tony Roberts, Carol Kane\",\"Plot\":\"Alvy Singer, a divorced Jewish comedian, reflects on his relationship with ex-lover Annie Hall, an aspiring nightclub singer, which ended abruptly just like his previous marriages.\",\"Language\":\"English, German\",\"Country\":\"USA\",\"Awards\":\"Won 4 Oscars. Another 26 wins & 8 nominations.\",\"Poster\":\"https://m.media-amazon.com/images/M/MV5BZDg1OGQ4YzgtM2Y2NS00NjA3LWFjYTctMDRlMDI3NWE1OTUyXkEyXkFqcGdeQXVyMjUzOTY1NTc@._V1_SX300.jpg\",\"Ratings\":[{\"Source\":\"Internet Movie Database\",\"Value\":\"8.0/10\"},{\"Source\":\"Rotten Tomatoes\",\"Value\":\"98%\"},{\"Source\":\"Metacritic\",\"Value\":\"92/100\"}],\"Metascore\":\"92\",\"imdbRating\":\"8.0\",\"imdbVotes\":\"252,992\",\"imdbID\":\"tt0075686\",\"Type\":\"movie\",\"DVD\":\"03 Sep 2015\",\"BoxOffice\":\"$38,251,425\",\"Production\":\"Rollins-Joffe Productions\",\"Website\":\"N/A\",\"Response\":\"True\"}"

In [ ]:
# Convert the JSON to a named R list.
content(resp)

# From JSON to R

## jsonlite

fromJSON() can convert character strings that represent JSON data into a nicely structured R list. 

In [16]:
# Load the jsonlite package
library(jsonlite)

# wine_json is a JSON
wine_json <- '{"name":"Chateau Migraine", "year":1997, "alcohol_pct":12.4, "color":"red", "awarded":false}'

# Convert wine_json into a list: wine
wine <- fromJSON(wine_json)

# Print structure of wine
str(wine)

List of 5
 $ name       : chr "Chateau Migraine"
 $ year       : int 1997
 $ alcohol_pct: num 12.4
 $ color      : chr "red"
 $ awarded    : logi FALSE


fromJSON() also works if you pass a URL as a character string or the path to a local file that contains JSON data. Let's try this out on the Quandl API, where you can fetch all sorts of financial and economical data.

## http link JSON

In [18]:
# jsonlite is preloaded

# Definition of quandl_url
quandl_url <- "https://www.quandl.com/api/v3/datasets/WIKI/FB/data.json?auth_token=i83asDsiWUUyfoypkgMz"

# Import Quandl data: quandl_data
quandl_data <- fromJSON(quandl_url)

# Print structure of quandl_data
str(quandl_data)

List of 1
 $ dataset_data:List of 10
  ..$ limit       : NULL
  ..$ transform   : NULL
  ..$ column_index: NULL
  ..$ column_names: chr [1:13] "Date" "Open" "High" "Low" ...
  ..$ start_date  : chr "2012-05-18"
  ..$ end_date    : chr "2018-03-27"
  ..$ frequency   : chr "daily"
  ..$ data        : chr [1:1472, 1:13] "2018-03-27" "2018-03-26" "2018-03-23" "2018-03-22" ...
  ..$ collapse    : NULL
  ..$ order       : NULL


In [19]:
class(quandl_data)

[1] "list"

##  Interacting with OMDb API

In [20]:
# Definition of the URLs
url_sw4 <- "http://www.omdbapi.com/?apikey=72bc447a&i=tt0076759&r=json"
url_sw3 <- "http://www.omdbapi.com/?apikey=72bc447a&i=tt0121766&r=json"

# Import two URLs with fromJSON(): sw4 and sw3
sw4 <- fromJSON(url_sw4)
sw3 <- fromJSON(url_sw3)

# Print out the Title element of both lists
sw4$Title
sw3$Title

# Is the release year of sw4 later than sw3?
sw4$Year > sw3$Year

[1] "Star Wars: Episode IV - A New Hope"

[1] "Star Wars: Episode III - Revenge of the Sith"

[1] FALSE